# jQAssistant Demo
* Clone https://github.com/JavaOnAutobahn/spring-petclinic
* Build software
 * `mvn install`
* Start Neo4j server
 * `mvn jqassistant:server`
* Open browser
 * http://localhost:7474/browser/
* jQAssistant documentation: https://jqassistant.github.io/jqassistant/doc/1.10.0/manual/index.html
 

## Example Cypher Queries

### Setup connection to a running N4o4j database

In [ ]:
from neo4j import GraphDatabase

URI = "bolt://localhost"
AUTH = ("neo4j", "neo4j")

driver = GraphDatabase.driver(URI, auth=AUTH)
driver.verify_connectivity()
session = driver.session()

### List TOP 10 class with the most methods

In [ ]:
query="""
MATCH 
  (t:Type)-[:DECLARES]->(m:Method)
RETURN t.fqn as Typ, COUNT(m) as methodCount
ORDER BY methodCount DESC
LIMIT 10
"""
session.run(query).to_df()

### Global  written variables

In [ ]:
query="""
MATCH (c:Class)-[:DECLARES]->(f:Field)<-[w:WRITES]-(m:Method)
WHERE 
    EXISTS(f.static) AND NOT EXISTS(f.final)
RETURN 
    c.name as InClass, 
    m.name as theMethod, 
    w.lineNumber as writesInLine, 
    f.name as toStaticField
"""
session.run(query).to_df()

### Recrusive data strutures

In [ ]:
query="""
MATCH (c:Class)-[:DECLARES]->(m:Method)-[:INVOKES]->(m)
RETURN c.name, m.name
"""
session.run(query).to_df()

### Aggregation of values across business subdomains

In [ ]:
query="""
MATCH 
  (t:Type)-[:BELONGS_TO]->(s:Business:Subdomain),
  (t)-[:HAS_CHANGE]->(ch:Change)
RETURN 
  s.name as ASubdomain,
  COUNT(DISTINCT t) as Types,
  COUNT(DISTINCT ch) as Changes
ORDER BY Types DESC
"""
session.run(query).to_df()

### Code dependencies between business subdomains

In [ ]:
query="""
MATCH
  (t1:Type)-[:BELONGS_TO]->(s1:Business:Subdomain),
  (t2:Type)-[:BELONGS_TO]->(s2:Business:Subdomain),
  (t1)-[d:DEPENDS_ON]->(t2)
WHERE
  s1 <> s2
WITH
  s1, s2, count(d) as weight
RETURN
  s1.name as Subdomain, collect(s2.name) as Dependencies
ORDER BY
  Subdomain
"""
session.run(query).to_df()

### Cyclic dependencies between packages

In [ ]:
query="""
MATCH
    (p1:Package)-[:DEPENDS_ON]->(p2:Package),
    path=shortestPath((p2)-[:DEPENDS_ON*]->(p1))
WHERE
    p1<>p2
RETURN
    p1 AS Package, EXTRACT(p IN nodes(path) | p.fqn) AS Cycle
ORDER BY
    Package.fqn
"""
session.run(query).to_df()